In [1]:
from pyspark.sql import SparkSession

In [2]:
import findspark
findspark.init()

In [3]:
spark = SparkSession.builder.appName("CSV to Parquet").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/12 09:57:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
import subprocess
p = subprocess.Popen("hdfs dfs -ls -d /parkingviolations/rawdata/* | awk '{print $8}' ",
                     shell=True,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.STDOUT)

csv_files = []

for line in p.stdout.readlines():
    csv_files.append(line.decode().strip())

p.wait()


0

In [5]:
#Now for the tricky part

In [22]:
for csv_file in csv_files:
    df = spark.read.csv(csv_file, header=True, inferSchema=True)
    file_name = csv_file.split("/")[-1].split(".")[0]
    parquet_file = f"/parkingviolations/parquet/{file_name}.parquet"
    df.write.parquet(parquet_file, mode="overwrite")

In [23]:
df = spark.read.parquet("/parkingviolations/parquet/Parking_Violations_Issued_-_Fiscal_Year_2014.parquet")

In [25]:
df.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

In [26]:
spark.stop()